# _Contents_

0. Inputs
2. Stratified smooth
2. Stratified wavy

## Initial and boundary conditions


In [ ]:
# Define Dirichlet boundary conditions
if any ([system == "system_1", system == "system_2"]): # Define Dirichlet boundary conditions for smooth flow
    if IBVP == "SS":
        variable1_dirichlet = Constant (ref[0])
        variable2_dirichlet = Constant (ref[1])
        variable3_dirichlet = Constant (ref[2])
        variable4_dirichlet = Constant (ref[3])
    elif IBVP == "SS_perturbed": # Define Dirichlet boundary conditions for perturbed flow 
        # Initial time
        timedirichlet = 0
        
        # Reference
        variable1_dirichlet = Constant (ref[0])
        variable2_dirichlet = Constant (ref[1])
        variable3_dirichlet = Constant (ref[2])
        variable4_dirichlet = Constant (ref[3])

        # Amplitude
        amplitude1 = (r[0]).real
        amplitude2 = (r[1]).real
        amplitude3 = (r[2]).real
        amplitude4 = (r[3]).real
        
        # Perturbation wave
        variable1_dirichlet = Expression ('variable1_dirichlet + amplitude1*sin (ang_frequency*timedirichlet - wavenumber*x[0])', 
                                            degree = p - 1, 
                                            variable1_dirichlet = variable1_dirichlet, 
                                            amplitude1 = amplitude1, 
                                            wavenumber  = wavenumber, 
                                            ang_frequency = ang_frequency, 
                                            timedirichlet = timedirichlet, 
                                            domain = mesh)
        variable2_dirichlet = Expression ('variable2_dirichlet + amplitude2*sin (ang_frequency*timedirichlet - wavenumber*x[0])', 
                                            degree = p, 
                                            variable2_dirichlet = variable2_dirichlet, 
                                            amplitude2 = amplitude2, 
                                            wavenumber  = wavenumber, 
                                            ang_frequency = ang_frequency, 
                                            timedirichlet = timedirichlet, 
                                            domain = mesh)
        variable3_dirichlet = Expression ('variable3_dirichlet + amplitude3*sin (ang_frequency*timedirichlet - wavenumber*x[0])', 
                                            degree = p, 
                                            variable3_dirichlet = variable3_dirichlet, 
                                            amplitude3 = amplitude3, 
                                            wavenumber  = wavenumber, 
                                            ang_frequency = ang_frequency, 
                                            timedirichlet = timedirichlet, 
                                            domain = mesh)
        variable4_dirichlet = Expression ('variable4_dirichlet + amplitude4*sin (ang_frequency*timedirichlet - wavenumber*x[0])', 
                                            degree = p - 1, 
                                            variable4_dirichlet = variable4_dirichlet, 
                                            amplitude4 = amplitude4, 
                                            wavenumber  = wavenumber, 
                                            ang_frequency = ang_frequency, 
                                            timedirichlet = timedirichlet, 
                                            domain = mesh)

elif system == "system_3":
    if IBVP == "SS":
        # Define Dirichlet boundary conditions for smooth flow
        variable1_dirichlet = Constant (ref[0]/rho_l)
        variable2_dirichlet = Constant (ref[2]/ref[0])
        variable3_dirichlet = Constant (ref[3]/ref[1])
        variable4_dirichlet = Constant (ref[1]/((1 - ref[0]/rho_l)*p_factor))

# Define vector of boundary conditions
if elementspace == "IPDG":
    # Define ds for facets
    # DOLFIN predefines the “measures” dx, ds and dS representing integration over cells, exterior facets (that is, facets on the boundary) and interior facets, respectively. These measures can take an additional integer argument. In fact, dx defaults to dx(0), ds defaults to ds(0), and dS defaults to dS(0). Integration over subregions can be specified by measures with different integer labels as arguments.
    # Define outer surface measure aware of Dirichlet boundaries
    ds = Measure ('ds', domain = mesh, subdomain_data = boundaries)
    # Dirichlet boundary conditions
    bc1 = DirichletBC (V.sub (0), variable1_dirichlet, boundaries, 1)#, "geometric") # "geometric"<<<<
    bc2 = DirichletBC (V.sub (1), variable2_dirichlet, boundaries, 1)#, "geometric")
    bc3 = DirichletBC (V.sub (2), variable3_dirichlet, boundaries, 1)#, "geometric")
    bc4 = DirichletBC (V.sub (3), variable4_dirichlet, boundaries, 2)#, "geometric")
    bcs = [bc1, bc2, bc3, bc4]
else:
    # Dirichlet boundary conditions
    bc1 = DirichletBC (V.sub (0), variable1_dirichlet, Inflow)#, "geometric") # "geometric" "pointwise"
    bc2 = DirichletBC (V.sub (1), variable2_dirichlet, Inflow)#, "geometric")
    bc3 = DirichletBC (V.sub (2), variable3_dirichlet, Inflow)#, "geometric")
    bc4 = DirichletBC (V.sub (3), variable4_dirichlet, Outflow)#, "geometric")
    bcs = [bc1, bc2, bc3, bc4]

# Initial conditions
if IBVP == 'SS':
    variable_init = InitialConditions_SS ()
elif IBVP == 'SS_perturbed':
    variable_init = InitialConditions_SS_perturbed ()

# interpolate vector of initial conditions
variable_ic.interpolate (variable_init)

# interpolate vector of initial conditions (transient simulations)
if time_method == 'BDF1':
    variable_n.interpolate (variable_init)
elif time_method == 'CN':
    variable_n.interpolate (variable_init)
elif time_method == 'BDF2':
    variable_past.interpolate (variable_init)

# Plot initial conditions
if any ([system == "system_1", system == "system_2"]):
    plt.figure (1, figsize = mapsize)
    fig, ax = plt.subplots ()
    # plt.ylim (0, 1)
    plt.xlim (0, L)
    plt.grid (True, which = "both")
    ax.set_xlabel (r'L [m]', fontsize = 18)
    ax.set_ylabel (r'$\alpha_l$ [-]', fontsize = 18)
    ax.xaxis.set_tick_params (which = 'major', size = 10, direction = 'in', top = 'on')
    ax.xaxis.set_tick_params (which = 'minor', size = 7, direction = 'in', top = 'on')
    ax.yaxis.set_tick_params (which = 'major', size = 10, direction = 'in', right = 'on')
    ax.yaxis.set_tick_params (which = 'minor', size = 7, direction = 'in', right = 'on')
    plot (variable1_ic, color = 'k')

    plt.savefig('figures/initial_conditions/fig1.pdf',
                optimize = True,
                transparent = True,  
                dpi = 1000)

    plt.figure (2, figsize = mapsize)
    fig, ax = plt.subplots ()
    plt.xlim (0, L)
    plt.grid (True, which = "both")
    ax.set_xlabel (r'L [m]', fontsize = 18)
    ax.set_ylabel (r'$u_l$ [m/s]', fontsize = 18)
    ax.xaxis.set_tick_params (which = 'major', size = 10, direction = 'in', top = 'on')
    ax.xaxis.set_tick_params (which = 'minor', size = 7, direction = 'in', top = 'on')
    ax.yaxis.set_tick_params (which = 'major', size = 10, direction = 'in', right = 'on')
    ax.yaxis.set_tick_params (which = 'minor', size = 7, direction = 'in', right = 'on')
    plot (variable2_ic, color = 'k')

    plt.savefig('figures/initial_conditions/fig2.pdf',
                optimize = True,
                transparent = True,  
                dpi = 1000)

    plt.figure (3, figsize = mapsize)
    fig, ax = plt.subplots ()
    plt.xlim (0, L)
    plt.grid (True, which = "both")
    ax.set_xlabel (r'L [m]', fontsize = 18)
    ax.set_ylabel (r'$u_g$ [m/s]', fontsize = 18)
    ax.xaxis.set_tick_params (which = 'major', size = 10, direction = 'in', top = 'on')
    ax.xaxis.set_tick_params (which = 'minor', size = 7, direction = 'in', top = 'on')
    ax.yaxis.set_tick_params (which = 'major', size = 10, direction = 'in', right = 'on')
    ax.yaxis.set_tick_params (which = 'minor', size = 7, direction = 'in', right = 'on')
    plot (variable3_ic, color = 'k')

    plt.savefig('figures/initial_conditions/fig3.pdf',
                optimize = True,
                transparent = True,  
                dpi = 1000)

    plt.figure (4, figsize = mapsize)
    fig, ax = plt.subplots ()
    plt.xlim (0, L)
    plt.grid (True, which = "both")
    ax.set_xlabel (r'L [m]', fontsize = 18)
    ax.set_ylabel (r'$p_i$ [Pa]', fontsize = 18)
    ax.xaxis.set_tick_params (which = 'major', size = 10, direction = 'in', top = 'on')
    ax.xaxis.set_tick_params (which = 'minor', size = 7, direction = 'in', top = 'on')
    ax.yaxis.set_tick_params (which = 'major', size = 10, direction = 'in', right = 'on')
    ax.yaxis.set_tick_params (which = 'minor', size = 7, direction = 'in', right = 'on')
    plot (variable4_ic, color = 'k')

    plt.savefig('figures/initial_conditions/fig4.pdf',
                optimize = True,
                transparent = True,  
                dpi = 1000)
else:
    pass